In [11]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
import json

# Read recipe inputs
d=json.loads("disk_usage.json")
disk_usage = dataiku.Dataset("disk_usage")
df = disk_usage.get_dataframe()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   status  1 non-null      object
 1   data    1 non-null      object
dtypes: object(2)
memory usage: 144.0+ bytes


In [8]:
df = str(df["data"])
type(df)

str

In [9]:
df = json.loads(df)

JSONDecodeError: Extra data: line 1 column 6 (char 5)

In [0]:
# Write recipe outputs
json = dataiku.Dataset("Json")
json.write_with_schema(df)